In [62]:
import tensorflow as tf
import keras
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


In [63]:
#LOAD THE TRAINED MODEL
model = keras.models.load_model(r"C:\Users\RIMSHA\ml\trainedModel.h5")

#DEFINE THE BACKGROUND AND WEIGHT
bg = None
accumulatedWeight = 0.5

#REGION OF INTEREST - DIMENSIONS
roiTop = 100
roiRight = 150
roiLeft = 350
roiBottom = 300

In [64]:
#TO DETECT OBJECTS IN THE FOREGROUND:
#CALCULATE ACCUMULATED AVERAGE FOR THE BACKGROUND
#CALCULATE ACCUMULATED WEIGHT FOR SOME FRAMES
#SUBTRACT THE ABOVE 2 -> FINDS ANY OBJECT

def calculateAccumulatedAverage(frame, accumulatedWeight):
    global bg
    if bg is None:
        bg = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, bg, accumulatedWeight)

In [65]:
def handSegmentation(frame, threshold=25):
    global bg
    #ABSDIFF -> ABSOLUTE DIFFERENCE OF THE BACKGROUND ACCUMULATED AVERAGE AND THE ACCUMULATED WEIGHT OF FRAME
    difference = cv2.absdiff(bg.astype("uint8"), frame)
    
    #THRESHOLD VALUE IS DEFINED
    #APPLY THRESHOLDING USING cv2.threshold function
    #SIMPLE THRESHOLDING (BINARY)
     #PIXEL VALUE < THRESHOLD => PIXEL VALUE = 0
     #PIXEL VALUE > THRESHOLD => PIXEL VALUE = MAX (THIRD PARAMETER (255 DEFINED IN THIS))
    ret, thresholded = cv2.threshold(difference, threshold, 255, cv2.THRESH_BINARY)
    
    #CONTOURS : a curve joining all the continuous points (along the boundary), having same color or intensity. The contours are a useful tool for shape analysis and object detection and recognition and size estimation.
    
    #GET CONTOURS IN THE FRAME
    #PARAMETERS -> thresholded image, contour retrieval model and contour approximation
    #it returns modified image
    #contours (list of contours -> each individual contour is numpy array)
    
    #NEXT 
    #the contour's length is checked to see if its a hand or not
    
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #no contours
    if len(contours) == 0:
        return None
    else:
    #hand contour would be the maximum one
        segmentedHandMaxContour = max(contours, key=cv2.contourArea)
        return (thresholded, segmentedHandMaxContour)
    
    

In [66]:
labelNames = {0:'Zero', 1:'One', 2:'two', 3:'Three', 4:'Four', 5:'Five', 6:'Six', 7:'Seven', 8:'Eight', 9:'Nine'}

#HAND DETECTION ON LIVE CAMERA
cam = cv2.VideoCapture(0)
numOfFrames = 0
while True:
    ret, frame = cam.read()
    
    #flip the frame for the inverted image captured
    frame = cv2.flip(frame, 1)
    
    frameCopy = frame.copy()
    
    #REGION OF INTEREST
    roi = frame[roiTop: roiBottom, roiRight: roiLeft]
    
    #CONVERT THE ROI FRAME INTO A GRAY FRAME THEN APPLY GAUSSIAN BLUR
    #TO GET B/W IMAGES
    grayF = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    if(numOfFrames < 70):
        calculateAccumulatedAverage(grayF, accumulatedWeight)
        cv2.putText(frameCopy, "FETCHING BACKGROUND...PLEASE WAIT", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    else:
        hand = handSegmentation(grayF)
        
        if hand is not None:
            thresholded, segmentedHand = hand
            cv2.drawContours(frameCopy, [segmentedHand + (roiRight, roiTop)], -1, (255,0,0), 1)
            cv2.imshow("Thresholded Hand Image", thresholded)
            
            thresholded = cv2.resize(thresholded, (128, 128))
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))
            
            prediction = model.predict(thresholded)
            cv2.putText(frameCopy, labelNames[np.argmax(prediction)], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    cv2.rectangle(frameCopy, (roiLeft, roiTop), (roiRight, roiBottom), (255,128,0), 3)
    numOfFrames += 1
    cv2.putText(frameCopy, "Sign Recognition" , (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign detection", frameCopy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()
        
            

1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 64ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 47ms/step
